## Imports

In [152]:
import pandas as pd
import re
import timeit
import seaborn as sns
import lightgbm as lgb
import numpy as np
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import sklearn.metrics as sm
from sklearn.ensemble import IsolationForest
import plotly.express as px
import numpy as np
import warnings
import datetime
import xgboost as xgb
import math
import joblib
import matplotlib.pyplot as plt
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
import tsaug
from tsaug.visualization import plot
import smogn
from catboost import CatBoostRegressor
import optuna.integration.lightgbm as lgb
import optuna


warnings.filterwarnings("ignore")

#Imports TFG Original
from prettytable import PrettyTable
from os import listdir
from os.path import isfile, join
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import accuracy_score , mean_squared_error
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from yellowbrick.model_selection import FeatureImportances
from sklearn.linear_model import LinearRegression



In [153]:
pathAltDynamic = "Resultats/GridSearchManual/AltTer/Dynamic Test/"
pathBaixDynamic = "Resultats/GridSearchManual/BaixTer/Dynamic Test/"
pathAltStatic = "Resultats/GridSearchManual/AltTer/Static Test/"
pathBaixStatic = "Resultats/GridSearchManual/BaixTer/Static Test/"

# Visualització Resultats (Alt Ter Dynamic Test)

In [154]:
normal = pd.read_csv(pathAltDynamic + 'AltTer-Normal-Dynamic.csv',index_col=0)
anomalies = pd.read_csv(pathAltDynamic + 'AltTer-Precipitacions-Dynamic.csv',index_col=0)
smogn = pd.read_csv(pathAltDynamic + 'AltTer-SMOGN-Dynamic.csv',index_col=0)



In [155]:
normal

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,13.613211,185.398983,8.894756,28.237992,308.635813
1,Random Forest,6.985548,120.726281,9.529280,27.250153,178.442123
2,Random Forest,16.672381,126.684400,9.439320,28.616363,193.656469
3,XGBoost,15.344536,180.783003,9.452795,30.016516,267.028930
4,XGBoost,8.661701,116.951909,10.287443,31.319900,133.241459
5,XGBoost,16.087979,114.396568,9.751827,32.863202,186.947163
6,LightGBM,15.808411,226.044968,8.631728,30.495464,323.217353
7,LightGBM,9.653445,204.646681,10.627121,28.287098,227.118006
8,LightGBM,23.731638,216.760354,9.834741,31.000720,250.858934
9,CatBoost,16.118436,230.771458,9.367525,32.134082,371.924378


In [156]:
anomalies

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,64.412252,179.437779,9.077250,35.086384,309.085750
1,Random Forest,40.976206,123.598435,8.355408,29.360511,185.281228
2,Random Forest,85.618925,134.725222,8.721956,32.233100,193.208559
3,XGBoost,62.482044,176.723715,9.641666,36.131299,270.711451
4,XGBoost,64.742657,118.995154,10.139152,35.041535,150.392294
5,XGBoost,82.328976,137.791330,9.693296,41.002016,168.466021
6,LightGBM,56.563754,155.460753,9.630863,37.214617,243.971293
7,LightGBM,68.800310,107.119870,9.461197,37.115416,169.422402
8,LightGBM,79.274979,130.175157,9.089364,36.127026,180.897733
9,CatBoost,62.245170,197.603179,11.314402,34.729828,273.722507


In [157]:
smogn

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,277.855603,224.960133,70.083243,45.952266,424.404526
1,Random Forest,36.460272,56.779954,7.014594,16.763133,44.726724
2,Random Forest,30.946228,52.515937,7.262854,11.958510,58.247745
3,XGBoost,271.721417,225.487143,61.108360,41.163716,416.141604
4,XGBoost,36.462895,66.317962,10.659880,14.975315,62.505260
5,XGBoost,35.261083,79.360683,8.575854,18.107646,50.451992
6,LightGBM,271.899126,220.412772,54.916325,38.071177,417.420150
7,LightGBM,36.642839,58.630740,9.664060,16.012545,49.500514
8,LightGBM,32.413759,53.305044,5.401524,16.785130,54.142411
9,CatBoost,270.159830,218.967343,36.200492,32.637664,416.241236


In [158]:
normal['Dataset'] = 'Normal'
anomalies['Dataset'] = 'Anomalies'
smogn['Dataset'] = 'SMOGN'

normal.index += 1
anomalies.index += 1
smogn.index += 1

normal = normal[normal.index % 3 == 0]  
anomalies = anomalies[anomalies.index % 3 == 0]  
smogn = smogn[smogn.index % 3 == 0]  

plot = pd.concat([normal, anomalies, smogn])

In [159]:
fig = px.histogram(plot, x="Method", y="RMSE Total",
             color='Dataset', barmode='group', title='RMSE Total (Alt Ter Dynamic Test)')
fig.show()

# Visualització Resultats (Alt Ter Static Test)

In [160]:
normal = pd.read_csv(pathAltStatic + 'AltTer-Normal-Static.csv',index_col=0)
anomalies = pd.read_csv(pathAltStatic + 'AltTer-Precipitacions-Static.csv',index_col=0)
smogn = pd.read_csv(pathAltStatic + 'AltTer-SMOGN-Static.csv',index_col=0)

In [161]:
normal

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,22.653274,180.512120,8.928869,27.122487,300.913915
1,Random Forest,15.868392,123.263667,9.447940,27.063623,175.438784
2,Random Forest,16.220407,126.855249,9.376924,27.163034,183.914919
3,XGBoost,22.288558,186.361240,10.717727,32.196245,286.496034
4,XGBoost,14.977009,113.946806,10.076744,32.203286,137.762200
5,XGBoost,16.087979,114.396568,9.751827,32.863202,186.947163
6,LightGBM,25.523460,226.044968,8.631728,30.495464,323.217353
7,LightGBM,22.925470,204.646681,10.627121,28.287098,227.118006
8,LightGBM,23.731638,216.760354,9.834741,31.000720,250.858934
9,CatBoost,26.115735,226.954777,10.609881,28.256942,338.407977


In [162]:
anomalies

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,121.442864,179.277369,8.943058,35.242440,303.350380
1,Random Forest,81.116693,121.617331,8.622131,30.336424,186.003719
2,Random Forest,84.317159,128.978819,8.762965,32.140176,190.120991
3,XGBoost,115.963889,176.723715,9.641666,36.131299,270.711451
4,XGBoost,81.185415,115.664007,10.978209,37.499185,145.558872
5,XGBoost,82.328976,137.791330,9.693296,41.002016,168.466021
6,LightGBM,104.686524,155.460753,9.630863,37.214617,243.971293
7,LightGBM,79.447474,107.119870,9.461197,37.115416,169.422402
8,LightGBM,79.274979,130.175157,9.089364,36.127026,180.897733
9,CatBoost,121.578363,194.331993,11.508513,34.747241,274.560346


In [163]:
smogn

,Method,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,147.898759,224.525790,69.994872,43.731298,421.297129
1,Random Forest,35.335584,54.252133,7.396048,17.079795,49.377183
2,Random Forest,31.401364,55.331176,7.124227,11.939162,55.731745
3,XGBoost,145.066433,222.385155,59.939747,46.444712,413.001640
4,XGBoost,36.122067,60.903491,9.582774,13.140057,55.011910
5,XGBoost,35.289500,58.610412,11.608670,13.318476,63.821280
6,LightGBM,144.732743,220.412772,54.916325,38.071177,417.420150
7,LightGBM,36.189316,58.630740,9.664060,16.012545,49.500514
8,LightGBM,32.413759,53.305044,5.401524,16.785130,54.142411
9,CatBoost,142.408359,219.086735,32.103040,29.313895,415.016464


In [164]:
normal['Dataset'] = 'Normal'
anomalies['Dataset'] = 'Anomalies'
smogn['Dataset'] = 'SMOGN'

normal.index += 1
anomalies.index += 1
smogn.index += 1

normal = normal[normal.index % 3 == 0]  
anomalies = anomalies[anomalies.index % 3 == 0]  
smogn = smogn[smogn.index % 3 == 0]  

plot = pd.concat([normal, anomalies, smogn])

In [165]:
fig = px.histogram(plot, x="Method", y="RMSE Total",
             color='Dataset', barmode='group', title='RMSE Total (Alt Ter Static Test)')
fig.show()

# Leslie (Alt Ter)

In [166]:
fig = px.histogram(plot, x="Method", y="RMSE_Leslie",
             color='Dataset', barmode='group', title='RMSE Leslie (Alt Ter)')
fig.show()

# Gloria (Alt Ter)

In [167]:
fig = px.histogram(plot, x="Method", y="RSME_Gloria",
             color='Dataset', barmode='group', title='RMSE Gloria (Alt Ter)')
fig.show()

# Visualització Resultats (Baix Ter Dynamic Test)

In [168]:
normal = pd.read_csv(pathBaixDynamic + 'BaixTer-Normal-Dynamic.csv',index_col=0)
anomalies = pd.read_csv(pathBaixDynamic + 'BaixTer-Precipitacions-Dynamic.csv',index_col=0)
smogn = pd.read_csv(pathBaixDynamic + 'BaixTer-SMOGN-Dynamic.csv',index_col=0)

In [169]:
normal

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,18.665472,73.972203,2.985590,112.256815,281.096927
1,Random Forest,NaN,4.599393,64.043164,2.202950,97.731000,210.857237
2,Random Forest,NaN,32.255884,43.631121,3.062366,113.618117,197.873400
3,XGBoost,NaN,17.862871,78.040279,3.814227,106.481505,296.954892
4,XGBoost,NaN,3.799999,73.491466,3.127113,146.605297,227.584657
5,XGBoost,NaN,35.323371,70.621340,2.027916,166.306446,239.832776
6,Linear Regression,NaN,18.712107,67.403826,6.593541,94.274869,293.215047
7,Linear Regression,NaN,5.248628,40.048537,5.202635,61.706282,305.764368
8,Linear Regression,NaN,43.806581,38.119600,5.018886,61.471407,292.413361
9,LightGBM,NaN,18.061830,77.471870,3.369236,103.166736,286.648262


In [170]:
anomalies

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,58.914712,117.713329,2.033182,130.765999,317.815379
1,Random Forest,NaN,56.740257,47.746128,2.417737,137.886366,283.720950
2,Random Forest,NaN,90.668085,40.567162,1.673339,115.843033,188.286499
3,XGBoost,NaN,59.969695,66.039035,3.730340,134.543090,325.267830
4,XGBoost,NaN,60.112065,149.077119,2.500338,162.909636,309.093123
5,XGBoost,NaN,98.883418,133.444445,2.054221,115.808111,226.489745
6,Linear Regression,NaN,128.897975,106.182179,6.533747,335.492031,651.364156
7,Linear Regression,NaN,55.323558,33.408738,24.343312,81.729688,159.896559
8,Linear Regression,NaN,131.157975,134.439515,19.545049,84.884613,449.281125
9,LightGBM,NaN,60.928831,69.079357,2.546119,139.110694,309.298734


In [171]:
smogn

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,266.854122,125.027233,57.281474,165.096129,410.251025
1,Random Forest,NaN,28.417861,25.185083,2.046382,99.741906,110.857745
2,Random Forest,NaN,23.666353,29.301259,1.940274,104.268014,98.991317
3,XGBoost,NaN,250.804818,110.517197,55.982861,161.819002,396.070263
4,XGBoost,NaN,30.217098,50.036224,2.574999,74.449174,51.923169
5,XGBoost,NaN,25.209671,57.128300,2.054390,102.479585,51.506695
6,Linear Regression,NaN,193.379645,60.586127,39.105620,116.269869,166.865796
7,Linear Regression,NaN,71.129226,43.545711,20.471142,62.735169,127.616988
8,Linear Regression,NaN,62.910071,41.661615,25.588310,62.301181,146.069715
9,LightGBM,NaN,251.958914,110.111144,42.213328,154.571692,391.242012


In [172]:
normal['Dataset'] = 'Normal'
anomalies['Dataset'] = 'Anomalies'
smogn['Dataset'] = 'SMOGN'

normal.index += 1
anomalies.index += 1
smogn.index += 1

normal = normal[normal.index % 3 == 0]  
anomalies = anomalies[anomalies.index % 3 == 0]  
smogn = smogn[smogn.index % 3 == 0]  

plot = pd.concat([normal, anomalies, smogn])

In [173]:
fig = px.histogram(plot, x="Method", y="RMSE Total",
             color='Dataset', barmode='group', title='RMSE Total (Baix Ter Dynamic Test)')
fig.show()

# Visualització Resultats (Baix Ter Static Test)

In [174]:
normal = pd.read_csv(pathBaixStatic + 'BaixTer-Normal-Static.csv',index_col=0)
anomalies = pd.read_csv(pathBaixStatic + 'BaixTer-Precipitacions-Static.csv',index_col=0)
smogn = pd.read_csv(pathBaixStatic + 'BaixTer-SMOGN-Static.csv',index_col=0)

In [175]:
normal

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,34.418417,57.645006,3.657765,106.741025,279.486400
1,Random Forest,NaN,33.775359,57.703792,2.384089,105.719085,219.684340
2,Random Forest,NaN,31.510456,45.690012,3.078341,100.021150,195.768318
3,XGBoost,NaN,19.039199,93.008238,4.128477,155.783589,300.401494
4,XGBoost,NaN,3.916324,74.221368,2.984512,148.745491,229.126811
5,XGBoost,NaN,38.248166,87.626360,2.712215,191.606390,228.072053
6,Linear Regression,NaN,57.672966,67.403826,6.593541,94.274869,293.215047
7,Linear Regression,NaN,44.934003,40.048537,5.202635,61.706282,305.764368
8,Linear Regression,NaN,43.806581,38.119600,5.018886,61.471407,292.413361
9,LightGBM,NaN,37.216590,77.471870,3.369236,103.166736,286.648262


In [176]:
anomalies

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,133.454943,95.789688,1.972637,126.716800,316.140349
1,Random Forest,NaN,116.805086,59.280829,2.383591,133.599499,293.756595
2,Random Forest,NaN,85.153913,35.802113,2.865771,128.623857,172.227166
3,XGBoost,NaN,59.969695,66.039035,3.730340,134.543090,325.267830
4,XGBoost,NaN,60.152902,148.497269,2.725845,162.721602,309.301063
5,XGBoost,NaN,104.864455,118.931791,2.300145,101.926247,221.743058
6,Linear Regression,NaN,274.083706,106.182179,6.533747,335.492031,651.364156
7,Linear Regression,NaN,86.826291,33.408738,24.343312,81.729688,159.896559
8,Linear Regression,NaN,131.157975,134.439515,19.545049,84.884613,449.281125
9,LightGBM,NaN,130.206893,69.079357,2.546119,139.110694,309.298734


In [177]:
smogn

,Method,RMSE_Score,RMSE Total,RMSE_Leslie,RMSE_Standard1,RMSE_Standard2,RSME_Gloria
0,Random Forest,NaN,145.382693,127.730008,66.629982,164.693635,411.258712
1,Random Forest,NaN,24.592621,24.525317,2.541367,94.688720,98.598430
2,Random Forest,NaN,23.710014,29.805857,2.065243,95.926167,96.737239
3,XGBoost,NaN,250.804818,110.517197,55.982861,161.819002,396.070263
4,XGBoost,NaN,30.449147,49.267899,3.463681,121.224609,45.552280
5,XGBoost,NaN,24.289113,55.115352,2.825589,120.265004,46.160202
6,Linear Regression,NaN,98.012044,60.586127,39.105620,116.269869,166.865796
7,Linear Regression,NaN,61.931054,43.545711,20.471142,62.735169,127.616988
8,Linear Regression,NaN,62.910071,41.661615,25.588310,62.301181,146.069715
9,LightGBM,NaN,135.435784,110.111144,42.213328,154.571692,391.242012


In [178]:
normal['Method'] = normal['Method'] + ' (Normal)'
anomalies['Method'] = anomalies['Method'] + ' (Anomalies)'
smogn['Method'] = smogn['Method'] + ' (SMOGN)'

normal.index += 1
anomalies.index += 1
smogn.index += 1

normal = normal[normal.index % 3 == 0]  
anomalies = anomalies[anomalies.index % 3 == 0]  
smogn = smogn[smogn.index % 3 == 0]  

In [179]:
fig = px.histogram(plot, x="Method", y="RMSE Total",
             color='Dataset', barmode='group', title='RMSE Total (Baix Ter Static Test)')
fig.show()

# Leslie (Baix Ter)

In [180]:
fig = px.histogram(plot, x="Method", y="RMSE_Leslie",
             color='Dataset', barmode='group', title='RMSE Leslie (Baix Ter)')
fig.show()

# Gloria (Baix Ter)

In [181]:
fig = px.histogram(plot, x="Method", y="RSME_Gloria",
             color='Dataset', barmode='group', title='RMSE Gloria (Baix Ter)')
fig.show()